In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import cnn               # CNN module
import pickle
from time import time

In [2]:
nn=cnn.conv_net()
w0,b0=nn.init_kernel_bias(num_inp_channels=3,kernel_size=3,num_kernels=64)

In [3]:
with open('saved_w8s.dump','rb') as f:
	w0,b0,w1,b1,w2,b2,w3,b3=pickle.load(f)

In [42]:
inp=np.random.randint(1,9,(40,3,3,3))

In [43]:
a=np.pad(inp,((0,0),(0,0),(1,1),(1,1)),'constant')

In [44]:
a.shape

(40, 3, 5, 5)

In [17]:
%timeit conv0=conv2d(1,inp,w0,b0)

In [18]:
conv0.shape

(40, 32, 32, 64)

In [88]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [19]:
conv0=conv2d(1,inp,w0,b0)

In [94]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col))
padded[:,:,padding:-padding,padding:-padding]=ipp

In [95]:
img=padded[0]

In [96]:
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
kern = w0.reshape(-1,num_ker)

In [97]:
img

array([[[0., 0., 0., 0., 0.],
        [0., 3., 2., 4., 0.],
        [0., 1., 3., 2., 0.],
        [0., 5., 4., 6., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 8., 7., 0.],
        [0., 1., 6., 4., 0.],
        [0., 6., 5., 3., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 8., 3., 6., 0.],
        [0., 5., 2., 4., 0.],
        [0., 3., 6., 7., 0.],
        [0., 0., 0., 0., 0.]]])

In [98]:
window

array([[ 0,  1,  2,  5,  6,  7, 10, 11, 12],
       [25, 26, 27, 30, 31, 32, 35, 36, 37],
       [50, 51, 52, 55, 56, 57, 60, 61, 62]])

In [99]:
slider

array([[ 0,  1,  2],
       [ 5,  6,  7],
       [10, 11, 12]])

In [ ]:
for img in padded:     #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    out=(np.dot(np.take(img,ind), kern))
    out=(out+biases).reshape(out_row,out_col,num_ker)
    output.append(out)

In [132]:
ind.shape

(9, 27)

In [61]:
padded.shape

(40, 3, 5, 5)

In [134]:
np.take(padded, ind).shape

(9, 27)

In [126]:
%timeit [np.take(i, ind) for i in padded]

82.4 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [129]:
(np.dot(np.take(img,ind), kern)).shape

(9, 64)